In [1]:
"""
    带有时间窗口约束的VRP问题
    1）将两点之间的distance转化为time
    2）添加时间约束 AddDimension
    3）还可以设置 车辆的装卸载时间
    4）还可以设置 起始仓库并发的 车辆数
"""
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """城市之间的行驶时间"""
    data = {}
    # 需要将两点之间的distance 转化为time
    data['time_matrix'] = [
        [0, 6, 9, 8, 7, 3, 6, 2, 3, 2, 6, 6, 4, 4, 5, 9, 7],
        [6, 0, 8, 3, 2, 6, 8, 4, 8, 8, 13, 7, 5, 8, 12, 10, 14],
        [9, 8, 0, 11, 10, 6, 3, 9, 5, 8, 4, 15, 14, 13, 9, 18, 9],
        [8, 3, 11, 0, 1, 7, 10, 6, 10, 10, 14, 6, 7, 9, 14, 6, 16],
        [7, 2, 10, 1, 0, 6, 9, 4, 8, 9, 13, 4, 6, 8, 12, 8, 14],
        [3, 6, 6, 7, 6, 0, 2, 3, 2, 2, 7, 9, 7, 7, 6, 12, 8],
        [6, 8, 3, 10, 9, 2, 0, 6, 2, 5, 4, 12, 10, 10, 6, 15, 5],
        [2, 4, 9, 6, 4, 3, 6, 0, 4, 4, 8, 5, 4, 3, 7, 8, 10],
        [3, 8, 5, 10, 8, 2, 2, 4, 0, 3, 4, 9, 8, 7, 3, 13, 6],
        [2, 8, 8, 10, 9, 2, 5, 4, 3, 0, 4, 6, 5, 4, 3, 9, 5],
        [6, 13, 4, 14, 13, 7, 4, 8, 4, 4, 0, 10, 9, 8, 4, 13, 4],
        [6, 7, 15, 6, 4, 9, 12, 5, 9, 6, 10, 0, 1, 3, 7, 3, 10],
        [4, 5, 14, 7, 6, 7, 10, 4, 8, 5, 9, 1, 0, 2, 6, 4, 8],
        [4, 8, 13, 9, 8, 7, 10, 3, 7, 4, 8, 3, 2, 0, 4, 5, 6],
        [5, 12, 9, 14, 12, 6, 6, 7, 3, 3, 4, 7, 6, 4, 0, 9, 2],
        [9, 10, 18, 6, 8, 12, 15, 8, 13, 9, 13, 3, 4, 5, 9, 0, 9],
        [7, 14, 9, 16, 14, 8, 5, 10, 6, 5, 4, 10, 8, 6, 2, 9, 0],
    ]
    data['time_windows'] = [
        (0, 5),  # depot
        (7, 12),  # 1
        (10, 15),  # 2
        (5, 14),  # 3
        (5, 13),  # 4
        (0, 5),  # 5
        (5, 10),  # 6
        (0, 10),  # 7
        (5, 10),  # 8
        (0, 5),  # 9
        (10, 16),  # 10
        (10, 15),  # 11
        (0, 5),  # 12
        (5, 10),  # 13
        (7, 12),  # 14
        (10, 15),  # 15
        (5, 15),  # 16
    ]
    data['num_vehicles'] = 4
    # 一辆车的装载时间
    data['vehicle_load_time'] = 5
    # 一辆车的卸载时间
    data['vehicle_unload_time'] = 5
    # 起始仓库容量：同一时刻，最大装载/卸载的 车辆数
    data['depot_capacity'] = 2
    data['depot'] = 0
    return data


# 输出路径规划结果
def print_solution(data, manager, routing, solution):
    # 输出总目标
    print(f'Objective: {solution.ObjectiveValue()}')
    time_dimension = routing.GetDimensionOrDie('Time')
    total_time = 0
    # 遍历所有车辆
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        while not routing.IsEnd(index):
            time_var = time_dimension.CumulVar(index)
            plan_output += '{0} Time({1},{2}) -> '.format(
                manager.IndexToNode(index), solution.Min(time_var),
                solution.Max(time_var))
            index = solution.Value(routing.NextVar(index))
        time_var = time_dimension.CumulVar(index)
        plan_output += '{0} Time({1},{2})\n'.format(manager.IndexToNode(index),
                                                    solution.Min(time_var),
                                                    solution.Max(time_var))
        plan_output += 'Time of the route: {}min\n'.format(
            solution.Min(time_var))
        print(plan_output)
        total_time += solution.Min(time_var)
    print('Total time of all routes: {}min'.format(total_time))


def main():
    # 初始化数据.
    data = create_data_model()

    # 创建路线管理，(邻接矩阵表的大小), num_vehicles（车的数量）, depot（原点）
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['depot'])

    # 创建Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # 两点之间行驶时间 回调函数
    def time_callback(from_index, to_index):
        # Convert from routing variable Index to time matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]
    
    # 创建距离回调函数，用于计算两个节点之间的距离
    transit_callback_index = routing.RegisterTransitCallback(time_callback)
    # 对所有的车辆，采用相同的time callback的计算
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # 添加时间窗口约束
    time = 'Time'
    routing.AddDimension(
        transit_callback_index,
        60,  # allow waiting time
        60,  # 每辆车最大时间
        False,  # Don't force start cumul to zero.
        time)
    # 获取Time约束
    time_dimension = routing.GetDimensionOrDie(time)
    
    # 对每个地点添加时间窗口约束（除了起始点） Add time window constraints for each location except depot.
    for location_idx, time_window in enumerate(data['time_windows']):
        # 起始点没有时间窗口约束
        if location_idx == 0:
            continue
        index = manager.NodeToIndex(location_idx)
        # 添加时间窗口约束
        time_dimension.CumulVar(index).SetRange(time_window[0], time_window[1])
        print(index, time_dimension.CumulVar(index))
        
    # 对每辆车的起始点，添加时间窗口约束 Add time window constraints for each vehicle start node.
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        time_dimension.CumulVar(index).SetRange(data['time_windows'][0][0],
                                                data['time_windows'][0][1])

    # Add resource constraints at the depot.
    solver = routing.solver()
    intervals = []
    for i in range(data['num_vehicles']):
        # 在路径起始添加时间窗口，没有固定的起始时间 Add time windows at start of routes
        intervals.append(
            solver.FixedDurationIntervalVar(
                time_dimension.CumulVar(routing.Start(i)),
                data['vehicle_load_time'], 'depot_interval'))
        # 在路径结束添加时间窗口 Add time windows at end of routes.
        intervals.append(
            solver.FixedDurationIntervalVar(
                time_dimension.CumulVar(routing.End(i)),
                data['vehicle_unload_time'], 'depot_interval'))
    print('intervals: ', intervals)
    print(len(intervals))
    # 每辆车装卸所需要的空间，这里所有的车辆都相等，默认都为1
    depot_usage = [1 for i in range(len(intervals))]
    print(depot_usage)
    # 添加约束，同一时间 最多两辆车
    solver.Add(
        solver.Cumulative(intervals, depot_usage, data['depot_capacity'],
                          'depot'))

    # 初始化路径的起始和结束时间，用于后续计算 Instantiate route start and end times to produce feasible times.
    for i in range(data['num_vehicles']):
        print('车辆{} 起始{} 终止{}'.format(i, routing.Start(i), routing.End(i)))
        print(time_dimension.CumulVar(routing.Start(i)))
        print(time_dimension.CumulVar(routing.End(i)))
        # 最小化
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.Start(i)))
        routing.AddVariableMinimizedByFinalizer(
            time_dimension.CumulVar(routing.End(i)))

    # 设置初始方案的 启发式方法
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # 问题求解
    solution = routing.SolveWithParameters(search_parameters)

    # 输出路径规划
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('No solution found !')

if __name__ == '__main__':
    main()

1 Time1(7..12)
2 Time2(10..15)
3 Time3(5..14)
4 Time4(5..13)
5 Time5(0..5)
6 Time6(5..10)
7 Time7(0..10)
8 Time8(5..10)
9 Time9(0..5)
10 Time10(10..16)
11 Time11(10..15)
12 Time12(0..5)
13 Time13(5..10)
14 Time14(7..12)
15 Time15(10..15)
16 Time16(5..15)
intervals:  [depot_interval(start = 0 .. 5, duration = 5, performed = true), depot_interval(start = 0 .. 60, duration = 5, performed = true), depot_interval(start = 0 .. 5, duration = 5, performed = true), depot_interval(start = 0 .. 60, duration = 5, performed = true), depot_interval(start = 0 .. 5, duration = 5, performed = true), depot_interval(start = 0 .. 60, duration = 5, performed = true), depot_interval(start = 0 .. 5, duration = 5, performed = true), depot_interval(start = 0 .. 60, duration = 5, performed = true)]
8
[1, 1, 1, 1, 1, 1, 1, 1]
车辆0 起始0 终止20
Time0(0..5)
Time20(0..60)
车辆1 起始17 终止21
Time17(0..5)
Time21(0..60)
车辆2 起始18 终止22
Time18(0..5)
Time22(0..60)
车辆3 起始19 终止23
Time19(0..5)
Time23(0..60)
Objective: 71
Route for veh